# Global Imports

In [ ]:
%matplotlib notebook
import concurrent.futures
import importlib
import os
import sys
from collections import Counter
from pprint import pprint

import imgaug.augmenters as iaa
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from skimage.color import label2rgb
from skimage.io import imread, imsave
from skimage.transform import resize
import pandas as pd

from mrcnn.model import MaskRCNN

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# Setup Data

In [ ]:
def setup_args():
    from keras.applications.resnet50 import preprocess_input
    from bidict import bidict
    from imgaug import augmenters as iaa
    from imgaug.parameters import Normal, Discretize
    from skimage.transform import resize
    
    def load_config(path):
        spec = importlib.util.spec_from_file_location(
            "maskrcnn_config", path)
        config_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(config_module)
        return config_module.Config()

#     def preprocess_data(image):
#         '''Transform the image before (possibly caching) and input to the network.'''
#        # This is done automatically by MRCNN

    def postprocess_data(image):
        '''Inverse transform of preprocess_data, used when trying to visualize images out of the dataset.'''
        return (image + 127.5).astype(np.uint8)

    def pipeline(gen, aug_config=None):
        '''The pipeline to run the dataset generator through.'''
        from abyss_deep_learning.keras.classification import multihot_gen, augmentation_gen

        return gen 
#                 (
#             augmentation_gen(
#                 multihot_gen(gen, num_classes=args['num_classes'])
#             , aug_config, enable=(aug_config is not None))
#         )

    augmentation_config = iaa.Sequential([ 
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.Affine(
            scale=(0.8, 1.2),
            translate_percent=(-0.2, 0.2), 
            rotate=(-22.5, 22.5),
            mode='constant', cval=0, order=0
        ),
        iaa.Sequential([ # Colour aug
            iaa.ChangeColorspace(from_colorspace="RGB", to_colorspace="HSV"),
            iaa.WithChannels(0, iaa.Add(Discretize(Normal(0, 256 / 6)))),
            iaa.WithChannels(1, iaa.Add(Discretize(Normal(0, 256 / 6)))),
            iaa.WithChannels(2, iaa.Add(Discretize(Normal(0, 256 / 6)))),
            iaa.ChangeColorspace(from_colorspace="HSV", to_colorspace="RGB")
        ])
    ])

    args = {
        'augmentation': None,#augmentation_config,    # Training augmentation
#         'caption_map': caption_map,             # Captio
        'data': {
            'base_dir': "/data/acfr/collated/2017-summer-lettuce",
            'name': "merged",
            'sets': ('train', 'val')
        },
        'config': load_config('/data/acfr/collated/2017-summer-lettuce/mrcnn_config.py'),
        'image_dims': (1024, 1024, 3),    # What to resize images to before CNN
        'nn_dtype': np.float32,         # Pretrained networks are in float32
        'num_classes': None,            # Calculate later
#         'use_balanced_set': False,      # Force the use of the largest class-balanced dataset
#         'use_cached': False,            # Cache the dataset in memory
#         'use_class_weights': True,      # Use class population to weight in the training loss
#         'use_parallel': False,          # Use multiple GPUs
#         'preprocess_data': preprocess_data,
        'postprocess_data': postprocess_data,
        'pipeline': pipeline
    }
    args['num_classes'] = args['config'].NUM_CLASSES
    
    return args
ARGS = setup_args()

# Setup Datasets

In [ ]:
def setup_datasets(args):
    from abyss_deep_learning.datasets.coco import MaskRcnnInstSegDataset
    
    dataset = dict()
    for set_name in args['data']['sets']:
        path = os.path.join(args['data']['base_dir'], "{:s}/{:s}.json".format(args['data']['name'], set_name))
        dataset[set_name] = MaskRcnnInstSegDataset(
            path, ARGS['config'])
        print("\n", set_name)
#         dataset[set_name].print_class_stats()

    print("\nNumber of classes:", args['num_classes'])
    cats = dataset['train'].coco.loadCats(dataset['train'].coco.getCatIds())
    class_names = ["BG"] + [
        cat['name'] for cat in sorted(cats, key=lambda x: x['id'])]
    print(class_names)
    return dataset, class_names

DATASET, ARGS['class_names'] = setup_datasets(ARGS)

In [ ]:
def display_from_inputs(inputs, **kwargs):
    from mrcnn.visualize import display_instances
    from mrcnn.utils import unmold_mask
    print(inputs[4].shape)
    N = np.argwhere(inputs[4][0] == 0)[0][0]
    image, image_meta = inputs[0][0], inputs[1][0]
    rpn_match, rpn_bbox = inputs[2][0], inputs[3][0]
    gt_class_ids, gt_boxes, gt_masks = inputs[4][0, :N], inputs[5][0, :N], inputs[6][0, ..., :N]

    masks = np.array([
        unmold_mask(gt_masks[..., idx], gt_boxes[idx], image.shape)
        for idx in range(N)]).transpose([1, 2, 0])

    display_instances(
        ARGS['postprocess_data'](image), gt_boxes, masks, gt_class_ids, ARGS['class_names'], **kwargs)
        
def view_dataset_samples(num_rows=2):
    plt.figure()
    print("Column-wise left to right, bottom row:")
    for i, (name, ds) in enumerate(DATASET.items()):
        print(name, end=' ')
        for j, (inputs, targets) in enumerate(ARGS['pipeline'](ds.mrcnn_generator(shuffle=True))):
            ax = plt.subplot(num_rows, 3, 3 * j + i + 1)
            display_from_inputs(inputs, ax=ax)
#             plt.title(', '.join([ARGS['caption_map'].inv[int(cap_id)] for cap_id in np.argwhere(label)]))
            plt.axis('off')
            if j + 1 == num_rows:
                break

view_dataset_samples(num_rows=2)

# Setup Model

In [ ]:
class Experiment(object):
    def __init__(self, config, model_dir):
        self.epoch = 0
        self.model = None
        self.config = config
        self.model_dir = model_dir
        self.compiled = False
    
    def create(self, model_path=None, train=False, fresh_heads=False, gpu_count=1):
        if not model_path:
            model_path = '/data/models/mask_rcnn_coco.h5'
            
        if not train:
            self.config.IMAGES_PER_GPU = 1
            self.config.BATCH_SIZE = 1
        self.model = None
        K.clear_session()
        self.config.GPU_COUNT = gpu_count
        self.model = MaskRCNN(
            mode=("training" if train else "inference"),
            config=self.config, model_dir=self.model_dir)
        if model_path: 
            exclude = [
                "mrcnn_class_logits", "mrcnn_bbox_fc",
                "mrcnn_bbox", "mrcnn_mask"] if fresh_heads else []
            self.model.load_weights(model_path, by_name=True, exclude=exclude)
        return self.model


# Create Inference MRCNN

In [ ]:
def scale_rois(rois, scale, max_height, max_width):
    h = rois[:,2] - rois[:,0]
    w = rois[:,3] - rois[:,1]
    rois += + np.round(np.vstack([-scale * h, -scale * w, scale * h, scale * w]).T).astype(np.int32)
    rois = np.maximum(0, rois)
    rois[:, 2] = np.minimum(max_height, rois[:, 2])
    rois[:, 3] = np.minimum(max_width, rois[:, 3])
    return rois

def save_result(path, image, r, threshold):
    '''note global OUTPUT_DIR'''
    basename = '.'.join(os.path.basename(path).split('.')[:-1])
#     image = imread(path)
    rois = scale_rois(r['rois'].copy(), 0.25, *image.shape[0:2])

    assert len(rois) == r['masks'].shape[-1] == len(r['class_ids']) == len(r['scores']), "shape mismatch"
    for plant_idx, (roi, mask, class_id, score) in enumerate(
            zip(rois, r['masks'].transpose([2, 0, 1]), r['class_ids'], r['scores'])):
        if score > threshold:
            output_path = os.path.join(OUTPUT_DIR, "{:s}_{:d}.png".format(basename, plant_idx))
            image_roi = image[roi[0]:roi[2], roi[1]:roi[3]]
            imsave(output_path, image_roi)
            output_path = os.path.join(OUTPUT_DIR, "{:s}_{:d}_mask.png".format(basename, plant_idx))
            image_roi = 255 * mask[roi[0]:roi[2], roi[1]:roi[3]]
            imsave(output_path, image_roi)

In [ ]:
output_dir = "/data/acfr/collated/thesis/cauliflower-broccoli"
OUTPUT_DIR = output_dir

!mkdir -p $output_dir
# model_weights = "/data/log/maskrcnn/weeks2to6/final2.h5"
model_weights = "/data/log/maskrcnn/broccoli-allages/merged/final.h5"
exp = None
ARGS['config'].USE_MINI_MASK = False
ARGS['config'].IMAGES_PER_GPU = 1
exp = Experiment(ARGS['config'], output_dir + "/log")
model = exp.create(model_path=model_weights, train=False, fresh_heads=False)

In [ ]:
from glob import glob
paths = [path 
    for date in ['201710', '201711', '201712']
    for path in glob("/data/acfr/ladybird/{:s}*/auto0/grasshopper3/img/left/*.png".format(date))
]

train_filenames = [img['file_name'] for img in DATASET['train'].coco.loadImgs(DATASET['train'].coco.getImgIds())]
paths = [path for path in paths if os.path.basename(path) not in train_filenames]
print("processing ", len(paths))
results = []

In [ ]:
results = []
for img_idx, path in enumerate(paths):
    image = imread(path)[np.newaxis, ...]
    basename = '.'.join(os.path.basename(path).split('.')[:-1])
    r = exp.model.detect(image)[0]
    num_dets = r['scores'].size
    if not num_dets:
        continue
    save_result(path, image[0], r, threshold=0.95)
    r['filename'] = [basename] * num_dets
    r['roi/y1'] = r['rois'][:, 0]
    r['roi/x1'] = r['rois'][:, 1]
    r['roi/y2'] = r['rois'][:, 2]
    r['roi/x2'] = r['rois'][:, 3]
    del r['rois'], r['masks']
    columns = [a for a in r.keys()]
#     print([a for a in r.values()])
    data = np.vstack([a for a in r.values()]).T
    df = pd.DataFrame(data, columns=columns)
    results.append(df)
#     plt.figure()
#     display_instances(
#         image[0],
#         r['rois'],
#         r['masks'],
#         r['class_ids'],
#         ARGS['class_names'], ax=plt.gca())

# def save_results(paths, results, threshold=0.975):
#     import concurrent.futures
# #     for path, result in zip(paths, results):
# #         save_result(path, result, threshold)
#     with concurrent.futures.ProcessPoolExecutor() as executor:
#         executor.map(save_result, paths, results, [threshold] * len(results))

# save_results(paths, results, threshold=0.5)
results = pd.concat(results).reset_index()
results.to_csv(os.path.join(OUTPUT_DIR, 'stats.csv'))

In [ ]:
valid = results[results['scores'].apply(float) >= 0.99]
valid

# Visualisation

In [ ]:
from mrcnn.utils import expand_mask
from mrcnn.visualize import display_images, display_instances
# from abyss_deep_learning.keras.segmentation import jaccard_index

def plot_test(gen, model, num_images=1, show=False):
    from scipy.optimize import linear_sum_assignment
    ious_list = []
    i = 0
    for ((images, image_meta, rpn_match, rpn_bbox, gt_class_ids, gt_boxes, gt_masks), targets) in gen:
        image = images[0]
        valid = np.all(gt_boxes[0], axis=1)
        class_ids = gt_class_ids[0, valid]
        masks = gt_masks[0, ..., valid].transpose((1, 2, 0))
        boxes = gt_boxes[0, valid, ...]
        
        labels = expand_mask(boxes, masks, image.shape).astype(np.uint8)
        r = model.detect([image], verbose=True)[0]
        num_pred = len(r['class_ids'])
        num_gt = len(class_ids)
        print("GTs = {:d}, Pred = {:d}".format(num_gt, num_pred))
        
        ious = np.array([[
            1 #jaccard_index(r['masks'][..., i] , labels[..., j]) 
                for j in range(labels.shape[-1])] 
                for i in range(r['masks'].shape[-1])])
        pred_idx, gt_idx = linear_sum_assignment(1-ious)
        r['ious'] = np.array([ious[pred_idx[i], gt_idx[i]] 
                              if (i in pred_idx and i in gt_idx) else 0.0 for i in range(num_pred)])
        print("IoUs", r['ious'])
        print("Scores", r['scores'])
        ious_list.append(ious)
        class_names = ['BG'] + [cat['name'] for cat in DATASET['train'].coco.cats.values()]
        if show:
            plt.figure()
            ax = plt.subplot(1, 2, 1)
            display_instances(
                image + ARGS['config'].MEAN_PIXEL,
                boxes,
                masks,
                class_ids,
                class_names, ax=ax)
            ax = plt.subplot(1, 2, 2, sharex=ax, sharey=ax)
            display_instances(
                image + ARGS['config'].MEAN_PIXEL,
                r['rois'],
                r['masks'],
                r['class_ids'],
                class_names, ax=ax)
            
#         imsave("/tmp/maskrcnn/image.png", (image + config.MEAN_PIXEL).astype(np.uint8))
        i += 1    
        if i >= num_images:
                break
    return ious_list

ious = plot_test(
    DATASET['val'].mrcnn_generator(shuffle=True),
    exp.model, num_images=1, show=True)

In [ ]:
import mrcnn.visualize as viz
# evaluate_coco(model, dataset_val, coco_val, eval_type="segm", limit=0, image_ids=None)
viz.display_weight_stats(exp.model)

In [ ]:
coco = DATASET['val']
image = coco.load_image(1)
exp
# Get activations of a few sample layers
activations = exp.model.run_graph([image], [
#     ("input_image",        exp.model.keras_model.get_layer("input_image").output),
    ("res2c_out",          exp.model.keras_model.get_layer("res2c_out").output),
    ("res3c_out",          exp.model.keras_model.get_layer("res3c_out").output),
    ("res4c_out",          exp.model.keras_model.get_layer("res4c_out").output),
    ("res5c_out",          exp.model.keras_model.get_layer("res5c_out").output),
    ("rpn_bbox",           exp.model.keras_model.get_layer("rpn_bbox").output),
    ("roi",                exp.model.keras_model.get_layer("ROI").output),
])

plt.figure()
layer_names = ["res2c_out", "res3c_out", "res4c_out", "res5c_out"]
ax = None
for i, layer in enumerate(layer_names):
    ax = plt.subplot(len(layer_names) // 2, 2, i + 1)
    plt.imshow(activations[layer].sum(axis=3)[0])
    plt.title(layer)
plt.tight_layout()

In [ ]:
# # Backbone feature map
# display_images(np.transpose(activations["res2c_out"][0,:,:,:4], [2, 0, 1]), cols=4)
# display_images(np.transpose(activations["res3c_out"][0,:,:,:4], [2, 0, 1]), cols=4)
# display_images(np.transpose(activations["res4c_out"][0,:,:,:4], [2, 0, 1]), cols=4)
# display_images(np.transpose(activations["res5c_out"][0,:,:,:4], [2, 0, 1]), cols=4)


# Experiment

In [ ]:
import gc
import itertools
import pickle

from herbicide.utils import vis_square
from skimage.io import imread
from skimage.transform import resize

In [ ]:
IMG_SIZE = [299, 299]
THUMB_SIZE = [50, 50]
IMAGES = None
FEATURES = None
MASKS = None

## Load Stats

In [ ]:
LOG_DIR = "/data/acfr/collated/thesis/lettuce"
def read_log_stats(logdir):
    size = pd.read_csv(
        os.path.join(logdir, "stats.csv"))
    size['width'] = size['roi/x2'] - size['roi/x1']
    size['height'] = size['roi/y2'] - size['roi/y1']
    size['ratio'] = size['height'] / size['width']
    size['area'] = size['height'] * size['width']
    size['ts'] = size['filename'].apply(pd.to_datetime)

    def between_times(ser, t1, t2):
        return np.logical_and(
            ser >= pd.to_datetime(t1), ser < pd.to_datetime(t2))
    # class0 = between_times(size['ts'], '20170329T020104.972891', '20170329T021056.206941')
    # class0 |= between_times(size['ts'], '20170329T021911.8', '20170329T023046.677172')
    class1 = between_times(size['ts'], '20170329T021056.206941', '20170329T021911.8')
    class1 |= between_times(size['ts'], '20170329T023046.677172', '20170329T023847.0')
    size['class_ids'] = np.ones_like(class1, dtype=np.uint8) + class1.astype(np.uint8)
    # size.rename()
    # size.index.name = 'index'
    size = size.drop(columns=['Unnamed: 0'])
    return size

def select_quantiles(df, quantiles, mask=None, exclude=[]):
    c = np.ones(len(df), dtype=np.bool)
    mask = mask if mask is not None else c.copy()
    for name in df.columns:
        if name not in exclude:
            c &= np.logical_and(
                df[name] > df[name][mask].quantile(quantiles[0]),
                df[name] < df[name][mask].quantile(quantiles[1]))
    return c

def drop_outliers(size):
    mask = (size['class_ids'] <= 1) #| (size['class_ids'] == 1)
    mask &= size['week_no'] == 0
    used = select_quantiles(size[['area', 'ratio']], [0.1, 0.9], mask=mask)
    used &= select_quantiles(size[['scores']], [0.2, 1.0], mask=mask)
    used &= mask
    # size.to_csv(os.path.join(LOG_DIR, 'stats.csv'))

    print(np.count_nonzero(used))
    plt.figure()
    plt.scatter(size['height'][~used], size['width'][~used], c=(0, 0.5, 0.5), alpha=0.1)
    plt.scatter(size['height'][used], size['width'][used], c=(0, 1, 0), alpha=0.3)
    plt.axis('equal')
    print(size['ts'][used].dt.week.value_counts())
    return size[used]

def sample_on(df, column):
    # Create views of 'pos' and 'neg' text.
    unique_labels = df[column].unique()
    num = STATS.groupby(column).size().min()
    
    # Equally sample 'pos' and 'neg' with replacement and concatenate into a dataframe.
    result = pd.concat([
        df[df[column] == label].sample(num)
        for label in unique_labels], axis=0)
    return result



STATS = read_log_stats(LOG_DIR)
STATS = STATS.rename({'index': 'idxz'}, axis=1)
STATS['week_no'] = STATS['ts'].dt.week.as_matrix()
STATS['week_no'] = STATS['week_no'] - STATS['week_no'].min()
STATS['class_ids'] = STATS['class_ids'] - 1
STATS = drop_outliers(STATS).reset_index(drop=True)

print(3*len(STATS) * (np.ones(IMG_SIZE + [4]).size + np.ones(THUMB_SIZE + [3]).size) / 1024 ** 3, "GB")

N_CLUSTERS = np.unique(STATS['class_ids']).size
print(N_CLUSTERS)
STATS['name'] = [
    "{:s}_{:d}.png".format(name, index)
    for name, index in zip(STATS['filename'], STATS['idxz'])]
print(len(STATS), STATS.index.max())

STATS.sample(10)

## Load Images

In [ ]:
def dataset_str(*args, **kwargs):
    return '-'.join(
        [item[1] for item in sorted(kwargs.items(), key=lambda x: x[0])]
    )
dataset_str(scale='stretch', mask='none', features='cnn-plantclef')

In [ ]:
from skimage.color import label2rgb
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_holes

def resize_image(image, shape, scale_mode='stretch', order=1):
    """mode can be 'stretch', 'square' or 'keep-scale'"""
    image_dtype = image.dtype
    if scale_mode == 'stretch':
        return resize(image, shape[0:2], mode='constant', cval=0, preserve_range=True, order=order).astype(image_dtype)
    if scale_mode == 'square':
        # resize largest side and pad smallest
        img_shape = image.shape
        scale = np.max(shape[:2]) / np.max(img_shape[:2])
        out_dims = np.array([img_shape[0] * scale, img_shape[1] * scale]).round().astype(int)
        image = resize(image, out_dims, mode='constant', cval=0, preserve_range=True, order=order).astype(image_dtype)
        
        img_shape = image.shape
        pad_y = (shape[0] - img_shape[0]) / 2
        pad_x = (shape[1] - img_shape[1]) / 2
        pad_y = [int(np.floor(pad_y)), int(np.ceil(pad_y))]
        pad_x = [int(np.floor(pad_x)), int(np.ceil(pad_x))]
        if any([i > 0 for i in pad_x + pad_y]):
            image = np.pad(
                image, [pad_y, pad_x, [0, 0]],
                mode='constant', constant_values=[[0], [0], [0]])
        return image
        
    if scale_mode == 'keep-scale':
        img_shape = image.shape
        diff_y = (shape[0] - img_shape[0]) / 2
        diff_x = (shape[1] - img_shape[1]) / 2
        diff_y = [int(np.floor(diff_y)), int(np.ceil(diff_y))]
        diff_x = [int(np.floor(diff_x)), int(np.ceil(diff_x))]

        # Crop
        crop_y = [
            abs(diff_y[0]) if diff_y[0] <= 0 else 0, 
            img_shape[0] - abs(diff_y[1]) if diff_y[1] <= 0 else img_shape[0]
        ]
        crop_x = [
            abs(diff_x[0]) if diff_x[0] <= 0 else 0, 
            img_shape[1] - abs(diff_x[1]) if diff_x[1] <= 0 else img_shape[1]
        ]
        image = image[crop_y[0]:crop_y[1], crop_x[0]:crop_x[1], ...]

        img_shape = image.shape
        pad_y = (shape[0] - img_shape[0]) / 2
        pad_x = (shape[1] - img_shape[1]) / 2
        pad_y = [int(np.floor(pad_y)), int(np.ceil(pad_y))]
        pad_x = [int(np.floor(pad_x)), int(np.ceil(pad_x))]
        if any([i > 0 for i in pad_x + pad_y]):
            image = np.pad(
                image, [pad_y, pad_x, [0, 0]],
                mode='constant', constant_values=[[0], [0], [0]])
        return image
    raise ValueError("prepare_image.resize_image bad scale_mode: " + str(scale_mode))
    
def prepare_image(name, modes):
    image_full = imread(os.path.join(LOG_DIR, name))
    image = dict()
    for mode in modes:
        if mode == 'thumb':
            image['thumb'] = resize(
                image_full, THUMB_SIZE, preserve_range=True, mode='constant', cval=0)
        else:
            image[mode] = resize_image(image_full, IMG_SIZE, scale_mode=mode, order=1)
    return image

def prepare_mask(name, images, idx, mask_modes, scale_modes):
    """Returns dict {tuple: image} where the tuple is (mask_mode, scale_mode)"""
    masks = dict()
    for key in itertools.product(mask_modes, scale_modes):
        mask_mode, scale_mode = key
        if scale_mode == 'thumb':
            continue
        if mask_mode == 'none':
            masks[key] = np.ones(IMG_SIZE + [1], dtype=np.uint8)
        elif mask_mode == 'mrcnn':
            mask_name = '.'.join(name.split('.')[:-1]) + "_mask." + name.split('.')[-1]
            image_full = imread(os.path.join(LOG_DIR, mask_name))
            if image_full.ndim == 2:
                image_full = image_full[..., np.newaxis]

            masks[key] = resize_image(image_full, IMG_SIZE, scale_mode=scale_mode, order=0)
        elif mask_mode == 'exgr':
            masks[key] = resize_image(
                exgr_mask(
                    images[scale_mode][idx]),
                IMG_SIZE, scale_mode=scale_mode, order=0)[..., np.newaxis]
        else:
            raise ValueError("prepare_mask unknown mask_mode '{}'".format(mask_mode))
    return masks

def load_images(mask_modes, scale_modes, load_batch_size=12, parallel=False):
    paths = STATS['name'].as_matrix().tolist()
    images = {
        mode: np.ones([len(paths), *IMG_SIZE, 3], dtype=np.uint8)
        for mode in scale_modes if mode != 'thumb'
    }
    masks = {
        (mask_mode, scale_mode): np.ones([len(paths), *IMG_SIZE, 1], dtype=np.uint8)
        for mask_mode, scale_mode in itertools.product(mask_modes, scale_modes)
        if scale_mode != 'thumb'
    }
    images['thumb'] = np.ones([len(paths), *THUMB_SIZE, 3], dtype=np.uint8)
    
    if parallel:
        raise NotImplementedError("Bug here no parallel")
        for b in range(0, len(paths), load_batch_size):
            e = b + np.minimum(load_batch_size, len(paths) - b)
            with concurrent.futures.ProcessPoolExecutor() as executor:
                for idx, ims in zip(range(b, e), executor.map(
                        prepare_image, paths[b:e], [scale_modes] * load_batch_size)):
                    for mode, image in ims.items():
                        images[mode][idx, ...] = image
    else:
        for idx, path in enumerate(paths):
            for scale_mode, image in prepare_image(path, scale_modes).items():
                images[scale_mode][idx, ...] = image
            for key, mask in prepare_mask(path, images, idx, mask_modes, scale_modes).items():
                masks[key][idx, ...] = mask
    return images, masks


def exgr_mask(image):
    image = image.astype(np.float64)
    image /= np.maximum(0, np.max(image, (0, 1)))
    image /= np.maximum(1, np.sum(image, 2)[..., np.newaxis])
    mask = (2 * image[..., 1] - image[..., 2] - image[..., 0]) > 0
    label_img = label(mask, background=0)
    idx = np.bincount(label_img.ravel())[1:].argmax() + 1 # choose the biggest blob
    mask = label_img == idx
    mask = remove_small_holes(mask).astype(np.uint8)
    return mask


In [ ]:
# STATS = STATS.sample(100).reset_index(drop=True)
SCALE_MODES = ['stretch', 'square', 'keep-scale']
MASK_MODES = ['none', 'mrcnn', 'exgr']

In [ ]:
IMAGES, MASKS = load_images(
    MASK_MODES, SCALE_MODES + ['thumb'],
    load_batch_size=24, parallel=False)

In [ ]:
for mask_type, scale_type in MASKS.keys():
    plt.figure()
#     plt.imshow(label2rgb(MASKS[(mask_type, scale_type)][0, ..., 0], IMAGES[scale_type][0]))
    plt.imshow((MASKS[(mask_type, scale_type)][0, ...] > 0) * IMAGES[scale_type][0])
    plt.title("{} {}".format(mask_type, scale_type))

## Load Features

In [ ]:
FEATURE_LIST = {
    "area": ("$\sqrt{area}$", lambda x: x ** 0.5, 'px'),
#     "bbox": ("bbox", lambda x: x, 'px'),
#     "bbox_area": ("$\sqrt{bbox\ area}$", lambda x: x ** 0.5, 'px'),
    "centroid": ("centroid", lambda x: x, 'px'),
    "convex_area": ("$\sqrt{convex\ area}$", lambda x: x ** 2, 'px'),
#     "convex_image": ("convex image", lambda x: x, 'px'),
#     "coords": ("coords", lambda x: x, 'px'),
    "eccentricity": ("eccentricity", lambda x: x, None),
    "equivalent_diameter": ("equivalent diameter", lambda x: x, 'px'),
#     "euler_number": ("euler number", lambda x: x, None),
#     "extent": ("extent", lambda x: x, None),
    "filled_area": ("$\sqrt{filled\ area}$", lambda x: x ** 0.5, 'px'),
#     "filled_image": ("filled image", lambda x: x, 'px'),
#     "image": ("image", lambda x: x, 'px'),
#     "inertia_tensor": ("inertia tensor", lambda x: x ** 0.5, '$px^2$'),
    "inertia_tensor_eigvals": ("inertia tensor eigvals", lambda x: x ** 0.5, '$px^2$'),
#     "intensity_image": ("intensity image", lambda x: x, None),
#     "label": ("label", lambda x: x, 'class number'),
    "local_centroid": ("local centroid", lambda x: x, 'px'),
    "major_axis_length": ("major axis length", lambda x: x, 'px'),
    "max_intensity": ("max intensity", lambda x: x, 'DN'),
    "mean_intensity": ("mean intensity", lambda x: x, 'DN'),
    "min_intensity": ("min intensity", lambda x: x, 'DN'),
    "minor_axis_length": ("minor axis length", lambda x: x, 'px'),
    "moments": ("moments", lambda x: x ** 0.25, '$px^4$'),
#     "moments_central": ("moments central", lambda x: x ** 0.5, '$DN\ \dot px^2$'),
#     "moments_hu": ("moments hu", lambda x: x, None),
#     "moments_normalized": ("moments normalized", lambda x: x, None),
    "orientation": ("orientation", lambda x: x, 'rad'),
    "perimeter": ("perimeter", lambda x: x, 'px'),
    "solidity": ("solidity", lambda x: x, None),
    "weighted_centroid": ("weighted centroid", lambda x: x ** 0.5, '$DN\ \dot px^2$'),
    "weighted_local_centroid": ("weighted local centroid", lambda x: x, '$DN\ \dot px$'),
    "weighted_moments": ("weighted moments", lambda x: x ** 0.5, '$DN\ \dot px^2$'),
#     "weighted_moments_central": ("$\sqrt{weighted\ moments\ central}$", lambda x: x ** 0.5, '$\sqrt{DN}\ \dot px$'),
#     "weighted_moments_hu": ("weighted moments hu", lambda x: x, '$DN$'),
#     "weighted_moments_normalized": ("weighted moments normalized", lambda x: x, '$DN$')
}

In [ ]:
def __extract_features_hcf(image, mask=None):
#         plt.figure()
#         plt.imshow(label2rgb(mask[..., 0] if mask.ndim == 3 else mask, image, bg_label=0))
        rp = regionprops(mask, image[..., 1])
        feats =  {
            feature_name: func(np.array(rp[0][feature_name])).ravel() \
            for feature_name, (_, func, _) in FEATURE_LIST.items()}
        feat_keys = np.hstack([
            ["{:s}/{:d}".format(key, i) for i, _ in enumerate(value)]
            for key, value in feats.items()])
        return np.hstack([feat for feat in feats.values()]), feat_keys

    
def extract_features(feature_mode, mask_mode, scale_mode):
    def extract_features_hcf(images, masks):
        import concurrent.futures
        assert masks is not None, 'hcf requires a valid mask'
        
        feats, feat_keys = __extract_features_hcf(images[0], masks[0])
        feats = np.ones([len(images), len(feats)])
        with concurrent.futures.ProcessPoolExecutor() as executor:
            for idx, feat in zip(range(len(feats)), executor.map(__extract_features_hcf, images, masks)):
                feats[idx, :] = feat[0]
        return feats, feat_keys

    def extract_features_convnet(images, masks, weights):
        from keras.applications.xception import Xception, preprocess_input
        from keras.models import model_from_json, Model
        from keras.layers import GlobalAveragePooling2D
#         from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

        K.clear_session()
        if weights == 'imagenet':
#             model = InceptionResNetV2(include_top=False, weights='imagenet', pooling='avg')
            model = Xception(include_top=False, weights='imagenet', pooling='avg')
        elif weights == 'plantclef':
            with open("/data/log/cnn/plantclef/xception/130epoch/model_def.json", "r") as file:
                model = model_from_json(file.read())
            model.load_weights(
                "/data/log/cnn/plantclef/xception/130epoch/model_weights.h5",
                by_name=False)
            model = Model(
                model.layers[1].inputs[0],
                GlobalAveragePooling2D()(model.layers[1].outputs[0])
                )
        features = np.array([
            model.predict(
                preprocess_input(image[np.newaxis, ...]) * (mask > 0)
            )[0]
            for image, mask in zip(images, masks)])
        return features, [str(i) for i in range(features.shape[1])]
    
    data = IMAGES[scale_mode]
    masks = MASKS[(mask_mode, scale_mode)]
    
    if feature_mode == 'hcf':
        X_gt_, feats_name = extract_features_hcf(data, masks)
    elif feature_mode == "cnn-plantclef":
        X_gt_, feats_name = extract_features_convnet(data, masks, weights='plantclef')
    elif feature_mode == "cnn-imagenet":
        X_gt_, feats_name = extract_features_convnet(data, masks, weights='imagenet')
    else:
        raise ValueError("extract_features: invalid f_type")
    return pd.DataFrame(X_gt_, columns=feats_name, index=STATS.index)

In [ ]:
## Test convnet features with various masks
def test_conv_masks(scale_name='stretch'):
    from keras.applications.xception import Xception, preprocess_input
    from keras.models import model_from_json, Model
    from keras.layers import GlobalAveragePooling2D

    with open("/data/log/cnn/plantclef/xception/130epoch/model_def.json", "r") as file:
            model = model_from_json(file.read())
    model.load_weights(
        "/data/log/cnn/plantclef/xception/130epoch/model_weights.h5",
        by_name=False)
    model = Model(
        model.layers[1].inputs[0],
        GlobalAveragePooling2D()(model.layers[1].outputs[0])
        )
    features = dict()
    image = IMAGES[scale_name][0]
    for mask_name in ['exgr', 'mrcnn', 'none']:
        mask = MASKS[(mask_name, scale_name)][0]
        masked_image = image * (mask > 0)
        print(image.shape, mask.shape, masked_image.shape)
        plt.figure()
        plt.imshow(masked_image)
        features[(mask_name, scale_name)] = model.predict(
            preprocess_input(masked_image[np.newaxis, ...]))[0]
    return features
    
feat_comp = test_conv_masks()
plt.figure()
plt.hist(
    [feat_comp[('none', 'stretch')].ravel(),
     feat_comp[('exgr', 'stretch')].ravel(),
     feat_comp[('mrcnn', 'stretch')].ravel()],
    bins=30, log=True);


In [ ]:
FEATURE_MODES =  ['hcf', 'cnn-plantclef', 'cnn-imagenet']
FEATURES = dict()

for comb in itertools.product(
        FEATURE_MODES,
        MASK_MODES,
        SCALE_MODES):
    if comb[0] == 'hcf' and comb[1] == 'none':
        continue
    print(comb)
    FEATURES[comb] = extract_features(*comb)

## Save/Load Images/Features

In [ ]:
save_name = "lettuce-type"
prefix = "/data/acfr/collated/thesis/lettuce/pickle/{:s}_".format(save_name)
# # IMAGES, MASKS, FEATURES = None, None, None

if IMAGES:
    with open(prefix + "IMAGES.pkz", "wb") as file:
        pickle.dump(IMAGES, file)
else:
    with open(prefix + "IMAGES.pkz", "rb") as file:
        IMAGES = pickle.load(file)

if MASKS:
    with open(prefix + "MASKS.pkz", "wb") as file:
        pickle.dump(MASKS, file)
else:
    with open(prefix + "MASKS.pkz", "rb") as file:
        MASKS = pickle.load(file)
        
if FEATURES:
    with open(prefix + "FEATURES.pkz", "wb") as file:
        pickle.dump(FEATURES, file)
else:
    with open(prefix + "FEATURES.pkz", "rb") as file:
        FEATURES = pickle.load(file)

## Test Classifier

In [ ]:
import sklearn.metrics
from mpl_toolkits.mplot3d import Axes3D
from sklearn.base import (BaseEstimator, ClassifierMixin, ClusterMixin,
                          TransformerMixin)
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering
from sklearn.decomposition import PCA, KernelPCA
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import (accuracy_score, classification_report,
                             precision_score)
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.model_selection import (GridSearchCV, StratifiedKFold,
                                     train_test_split)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.utils.class_weight import compute_class_weight
from sklearn.neighbors import kneighbors_graph


# Setup data
stats = sample_on(STATS, 'class_ids')
features = FEATURES[('cnn-imagenet', 'exgr', 'stretch')]
features = features.loc[stats.index & features.index]

class_key = 'class_ids'
X_gt = features
y_gt = stats[class_key]#[subset]
print("shapes:", X_gt.shape, y_gt.shape)
print("types:", type(X_gt), type(y_gt))
print("classes:", np.unique(y_gt))

# Data examples
def test(stats, y_gt, class_key):
    unique_y = np.unique(y_gt)
    plt.figure()
    for i, label in enumerate (unique_y):
        plt.subplot(1, unique_y.size, 1 + i)
        idx = stats[stats[class_key] == label].sample().index
        plt.imshow(IMAGES['stretch'][idx[0].astype(int), ...])
test(stats, y_gt, class_key)


### PCA Plot

In [ ]:
# PCA Plot
def plot_labels(X, y_true, y_pred, pca=True):
    markers = np.array(['C'+str(i) + '.+v^<>s8p*xD'[j] for i, j in zip(y_true, y_pred)])
    pca_feats = PCA(n_components=3).fit_transform(
        StandardScaler(with_mean=True, with_std=True).fit_transform(X))
    X = pca_feats if pca else X
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    for marker in np.unique(markers):
        pos = X[markers == marker]
        ax.plot3D(pos[:, 0], pos[:, 1], pos[:, 2], marker, alpha=0.5)
#     ax.scatter(pca_feats[:, 0], pca_feats[:, 1], pca_feats[:, 2], c=c)
    print(sorted(Counter(markers).items(), key=lambda x: x[0]))
    
    
plot_labels(X_gt, y_gt, y_gt)

### Class histograms

In [ ]:
plt.figure()
for label in np.unique(y_gt):
    X_class = X_gt[y_gt == label]
    summary = X_class.mean(axis=0)
    print(X_class.shape, summary.shape)
    plt.plot(summary, '.')

### Test classification

In [ ]:
skf = StratifiedKFold(n_splits=3, shuffle=True)
for split_no, (train_index, test_index) in enumerate(skf.split(X_gt, y_gt)):
    X_train, X_test = X_gt.iloc[train_index], X_gt.iloc[test_index]
    y_train, y_test = y_gt.iloc[train_index], y_gt.iloc[test_index]
    class_weight = compute_class_weight('balanced', sorted(np.unique(y_train)), y_train)
    class_weight = {label: weight for label, weight in zip(sorted(np.unique(y_train)), class_weight)}
    scaler = StandardScaler(with_mean=True, with_std=True)
    estimator = SVC(C=1.0, class_weight=class_weight, probability=True)
    
    estimator.fit(scaler.fit_transform(X_train), y_train)
    y_pred = estimator.predict_proba(scaler.transform(X_test))
    accuracy = sklearn.metrics.accuracy_score(y_test, y_pred.argmax(-1))
    average_precision = sklearn.metrics.average_precision_score(y_test, y_pred.argmax(-1), average='micro')
    print('Average precision-recall score: {0:0.2f}'.format(average_precision))
    print('Accuracy score: {0:0.2f}'.format(accuracy))
    
precision, recall, _ = sklearn.metrics.precision_recall_curve(
    y_test, y_pred[:,1])

plt.figure()
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

plot_labels(X_test, y_test, y_pred.argmax(-1))

### Test Clustering

In [ ]:
from sklearn.neural_network import BernoulliRBM
from sklearn.preprocessing import minmax_scale

In [ ]:
# Standard Clustering
skf = StratifiedKFold(n_splits=3, shuffle=True)
for split_no, (train_index, test_index) in enumerate(skf.split(X_gt, y_gt)):
    X_train, X_test = X_gt.iloc[train_index], X_gt.iloc[test_index]
    y_train, y_test = y_gt.iloc[train_index], y_gt.iloc[test_index]

#     plt.figure()
#     plt.hist(X_train.mean(axis=-1))
#     print(X_train[0].shape)
    
    scaler = StandardScaler(with_mean=False, with_std=False)
    transformer = StandardScaler(with_mean=False, with_std=False)
#     transformer = KernelPCA(n_components=3, kernel='rbf', gamma=1e-2)
    transformer = PCA(n_components=3)
#     transformer = BernoulliRBM(n_components=3, learning_rate=1e-4, n_iter=10)
    
#     estimator = BayesianGaussianMixture(
#         n_components=10,
#         weight_concentration_prior_type="dirichlet_process",
#         weight_concentration_prior=1e1)
    estimator = KMeans(n_clusters=2)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    X_train, X_test = minmax_scale(X_train), minmax_scale(X_test)
    
    X_train = transformer.fit_transform(X_train)
    X_test = transformer.transform(X_test)

    estimator.fit(X_train)
    y_pred = estimator.predict(X_test)
    
    print(sklearn.metrics.normalized_mutual_info_score(y_test, y_pred))
#     print(estimator.weights_)
    print(np.unique(y_pred))
    
plot_labels(X_test, y_test, y_pred, pca=True)

In [ ]:
import IPython.display

def display_table(table):
    """Display values in a table format.
    table: an iterable of rows, and each row is an iterable of values.
    """
    html = ""
    for row in table:
        row_html = ""
        for col in row:
            row_html += "<td>{:40}</td>".format(str(col))
        html += "<tr>" + row_html + "</tr>"
    html = "<table>" + html + "</table>"
    IPython.display.display(IPython.display.HTML(html))


def display_weight_stats(model):
    """Scans all the weights in the model and returns a list of tuples
    that contain stats about each weight.
    """
    layers = model.layers
    table = [["WEIGHT NAME", "TRAIN", "SHAPE", "MIN", "MAX", "STD"]]
    for l in layers:
        weight_values = l.get_weights()  # list of Numpy arrays
        weight_tensors = l.weights  # list of TF tensors
        for i, w in enumerate(weight_values):
            weight_name = weight_tensors[i].name
            # Detect problematic layers. Exclude biases of conv layers.
            alert = ""
            if w.min() == w.max() and not (l.__class__.__name__ == "Conv2D" and i == 1):
                alert += "<span style='color:red'>*** dead?</span>"
            if np.abs(w.min()) > 1000 or np.abs(w.max()) > 1000:
                alert += "<span style='color:red'>*** Overflow?</span>"
            # Add row
            table.append([
                weight_name + alert,
                str(weight_tensors[i].trainable),
                str(w.shape),
                "{:+9.4f}".format(w.min()),
                "{:+10.4f}".format(w.max()),
                "{:+9.4f}".format(w.std()),
            ])
    display_table(table)

import keras.models

model = None
K.clear_session()
with open("/data/log/cnn/plantclef/xception/130epoch/model_def.json", "r") as file:
    model = keras.models.model_from_json(file.read())
model.load_weights(
    "/data/log/cnn/plantclef/xception/130epoch/model_weights.h5",
    by_name=False)



In [ ]:
# model.summary()
display_weight_stats(model)

In [ ]:
# Agglom Clustering

skf = StratifiedKFold(n_splits=3, shuffle=True)
for split_no, (train_index, test_index) in enumerate(skf.split(X_gt, y_gt)):
    X_train, X_test = X_gt.iloc[train_index], X_gt.iloc[test_index]
    y_train, y_test = y_gt.iloc[train_index], y_gt.iloc[test_index]
    knn_graph = kneighbors_graph(X_test, 30, include_self=False)
    scaler = StandardScaler(with_mean=True, with_std=True)
    transformer = StandardScaler(with_mean=False, with_std=False)
    transformer = PCA(n_components=3)
    
    estimator = AgglomerativeClustering(
        linkage='ward',
        connectivity=knn_graph,
        n_clusters=2)
    estimator.fit(transformer.fit_transform(scaler.fit_transform(X_test)))
    y_pred = estimator.labels_
    
    print(sklearn.metrics.normalized_mutual_info_score(y_test, y_pred))
#     print(estimator.weights_)
    print(np.unique(y_pred))
plot_labels(X_test, y_test, y_pred, pca=True)

In [ ]:
# pca_w = np.linalg.norm(pipeline.named_steps.feat_sel.components_, axis=0)
# pca = pd.DataFrame(
#     data=pipeline.named_steps.feat_sel.explained_variance_ratio_,  
#     index=pd.Series([feats_name[i] for i in pca_w.argsort()[::-1]], name='Feature'),
#     columns=['Var Ratio'])
# print(pca.sum())
# pca


In [ ]:
# # Test Classifier


# def test_classifier(feature_set, weeks):
#     subset = STATS['week_no'].apply(lambda x: x in weeks).as_matrix()
#     X_train, X_test, y_train, y_test = train_test_split(
#         FEATURES[feature_set][0][subset],
#         STATS['week_no'][subset],
#         test_size=0.2)
#     class_weight = compute_class_weight('balanced', sorted(np.unique(y_train)), y_train)
#     class_weight = {label: weight for label, weight in zip(sorted(np.unique(y_train)), class_weight)}
#     estimator = Pipeline([
#         ('scaler', StandardScaler()),
# #         ('transformer', PCA(n_components=30)),
#         ('estimator', SVC(kernel='rbf', C=1, class_weight=class_weight))
#     ])

#     estimator.fit(X_train, y_train)
#     return estimator.score(X_test, y_test)


# # import statsmodels.api as sm
# # df = pd.DataFrame(results)
# # plt.figure()
# # for i, (supervision, group) in enumerate(df.groupby('supervision')):
# #     ax = plt.subplot(1, 2, 1 + i)
# #     vals = group.sort_values(by='k_features')
# #     vals.plot(x='k_features', y='precision', style='.', ax=ax)
# #     regression = sm.formula.ols(formula='precision ~ k_features', data=vals)
# #     res = regression.fit()
# #     vals.assign(fit=res.fittedvalues).plot(x='k_features', y='fit', ax=ax)
# # plt.suptitle('HCF Features')
# # plt.gcf().axes[0].set_ylabel('precision')
# # # df[['precision', 'supervision']].boxplot( by='supervision')

# results = []
# _weeks = [(0, 1), (0, 1, 2), (0, 1, 2, 3), (0, 1, 2, 3, 4)]
# for features, image_mode in FEATURES.keys():
#     print(features, image_mode)
#     for weeks in _weeks:
#         score = test_classifier((features, image_mode, mask_mode), weeks)
#         results.append({
#             'features': features + "-" + mask_mode if features == "hcf" else features,
#             'image scaling': image_mode,
#             'weeks': weeks[-1],
#             'test score': score
#         })
# results = pd.DataFrame(results)
# results.groupby(['features', 'image scaling', 'weeks']).mean()


In [ ]:
# plt.figure(figsize=(10,6), dpi=150)
# _, axes = plt.subplots(1, 2)
# for i, (key, group) in enumerate(results.groupby("features")):
#     ax = 0 if key == 'cnn' else 1
#     for scale_type, subgroup in group.groupby('image scaling'):
#         subgroup.sort_values('weeks').plot(
#             x='weeks', y='test score',
#             label=str( key + " " + scale_type), ax=axes[ax])
    


In [ ]:
# sample = STATS[['height', 'width', 'filename']][STATS['week_no'] == 4].sample()['filename']
# print(IMAGES['keep-scale'][sample.index].shape)
# plt.figure()
# plt.subplot(1, 3, 1)
# plt.imshow(IMAGES['keep-scale'][sample.index][0])
# plt.subplot(1, 3, 2)
# plt.imshow(exgr_mask(IMAGES['keep-scale'][sample.index][0]))
# plt.subplot(1, 3, 3)
# plt.imshow(MASKS['keep-scale'][sample.index][0][..., 0])

In [ ]:
def images_to_sprite(data):
    """Creates the sprite image along with any necessary padding

    Args:
      data: NxHxW[x3] tensor containing the images.

    Returns:
      data: Properly shaped HxWx3 image with any necessary padding.
    """
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))
    data = data.astype(np.float32)
    min = np.min(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) - min).transpose(3,0,1,2)
    max = np.max(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) / max).transpose(3,0,1,2)
    # Inverting the colors seems to look better for MNIST
    #data = 1 - data

    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0),
            (0, 0)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant',
            constant_values=0)
    # Tile the individual thumbnails into an image.
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3)
            + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    data = (data * 255).astype(np.uint8)
    return data

def save_embeddings(LOG_DIR, features, labels, sprite=None, sprite_shape=None):
    import os
    import tensorflow as tf
    from tensorflow.contrib.tensorboard.plugins import projector
    
    !rm -R "$LOG_DIR"
    !mkdir -p "$LOG_DIR"

    metadata = os.path.join(LOG_DIR, 'metadata.tsv')
    features = tf.Variable(features, name='features')

    with open(metadata, 'w') as metadata_file:
        for row in labels:
            metadata_file.write('%d\n' % row)
            
    if sprite is not None:
        imsave(os.path.join(LOG_DIR, 'sprite.png'), sprite)

    with tf.Session() as sess:
        saver = tf.train.Saver([features])

        sess.run(features.initializer)
        saver.save(sess, os.path.join(LOG_DIR, 'features.ckpt'))

        config = projector.ProjectorConfig()
        # One can add multiple embeddings.
        embedding = config.embeddings.add()
        embedding.tensor_name = features.name
        # Link this tensor to its metadata file (e.g. labels).
        embedding.metadata_path = metadata
        if sprite is not None:
            embedding.sprite.image_path = os.path.join(LOG_DIR, 'sprite.png')
            embedding.sprite.single_image_dim.extend(sprite_shape)
        # Saves a config file that TensorBoard will read during startup.
        projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)


In [ ]:

# save_embeddings(
#     "/data/log/embeddings/lettuce-w2to6-HCFcrop_SS_PCA_RBF_GMM",
#     StandardScaler().fit_transform(X_gt_),
#     y_gt_,
#     sprite=images_to_sprite(thumbs), sprite_shape=THUMB_SIZE)
## d = pd.DataFrame(X_gt_, columns=feats_name)
## d.to_csv("/data/acfr/collated/thesis/hcf-crop.csv")

## Full Experiment

In [ ]:
class Noop(BaseEstimator, TransformerMixin):
    def fit_transform(self, X, y=None, **fit_params):
        return X
    def fit(self, X, y=None, **kwargs):
        pass
    def transform(self, X, **kwargs):
        return X
    def set_params(self, *args, **kwargs):
        pass

class ClassificationExperiment(BaseEstimator, TransformerMixin):
    """
    Parameters:

        scaling_type: one of the following:
            'stretch': Stretch the image across image_size, does not keep ratio.
            'square': Stretch the largest dimension, pad the smallest to keep h/w ratio.
            'keep-scale': Pad or crop both dimensions to keep the x/y ratio.
            
        image_size: [N, M] shape to resize images to #TODO
        
        features: one of the following:
            'hcf-exgr': Use selected hand crafted features (ExGR mask)
            'hcf-mrcnn': Use selected hand crafted features (MRCNN masks)
            'cnn-imagenet': Use CNN features trained from imagenet
            'cnn-plantclef': Use CNN features trained from plantCLEF
        
        cnn_pooling: either 'avg' or 'max'
    """
    def __init__(
            self, features='cnn-plantclef', mask_mode='exgr', scaling_type='stretch'):
        self.scaling_type = scaling_type
        self.features = features
        self.mask_mode = mask_mode
#         self.image_size = image_size
#         self.cnn_pooling = cnn_pooling
        self._reload_data()
        
    def _reload_data(self):
        key = (self.features, self.mask_mode, self.scaling_type)
        if key not in FEATURES:
            raise ValueError("Not a valid combination of scaling_type and features " + str(key))
        self.X_gt_ = FEATURES[key]
#         self.X_gt_ = self.X_gt_
        self.y_gt_ = STATS['class_ids'].as_matrix()
        
    def fit(self, *args, **kwargs):
        return self
    
    def transform(self, X, y=None, **kwargs):
        """x is indices of features and labels"""
        return self.X_gt_.iloc[X]
    
#     def transform_label(self, y, **kwargs):
#         """x is indices of features and labels"""
#         return self.y_gt_.iloc[y]
    
    def score(self, X, y=None):
        return 0.0
    
    def fit_transform(self, X, y=None, **kwargs):
        """x is indices of features and labels"""
        return self.transform(X, y)
    
    def stats(self, test_size=0.1):
        X_train, X_test, y_train, y_test = train_test_split(self.X_gt_, self.y_gt_, test_size=test_size)
        print("trivial accuracy")
        print([np.count_nonzero(l == y_test) / y_test.size for l in sorted(np.unique(y_test))])
        
    def set_params(self, **params):
        reload_data = False
        if 'features' in params:
            self.features = params.pop('features')
            reload_data = True
        if 'scaling_type' in params:
            self.scaling_type = params.pop('scaling_type')
            reload_data = True
        if reload_data:
            self._reload_data()
        super().set_params(**params)

In [ ]:
exp = ClassificationExperiment(
    scaling_type='stretch', mask_mode='none', features='cnn-plantclef')
exp.stats()
exp.get_params()

In [ ]:
# N_CLUSTERS = 2
from sklearn import metrics
def encode_onehot(x, num_classes):
    y = np.zeros((len(x), num_classes))
    y[np.arange(len(x)), x] = 1
    return y

def micro_average_precision_score(x, y):
    return metrics.average_precision_score(
        encode_onehot(x, N_CLUSTERS),
        encode_onehot(y, N_CLUSTERS),
        average='micro')

y_train = STATS['class_ids'].as_matrix()
class_weight = compute_class_weight('balanced', sorted(np.unique(y_train)), y_train)
class_weight = {label: weight for label, weight in zip(sorted(np.unique(y_train)), class_weight)}

In [ ]:
def experiment(experiment_type):
    feature_modes =  ['hcf']#'cnn-plantclef', 'cnn-imagenet']
    mask_modes = ['exgr', 'mrcnn']
    if experiment_type == 'classification':
        parameters = [ # CLASSIFICATION
            {
                'source__features': feature_modes,
                'source__scaling_type': SCALE_MODES,
                'source__mask_mode': mask_modes,
                'transformer': [None],
                'estimator': [LinearSVC(penalty='l1', dual=False, class_weight=class_weight)],
                'estimator__C': 2.0 ** np.arange(-4, 2),
            },
            {
                'source__features': feature_modes,
                'source__scaling_type': SCALE_MODES,
                'source__mask_mode': mask_modes,
                'transformer': [None],
                'estimator': [SVC(kernel='rbf', class_weight=class_weight)],
                'estimator__C': 2.0 ** np.arange(-1, 5),
            },
            {
                'source__features': feature_modes,
                'source__scaling_type': SCALE_MODES,
                'source__mask_mode': mask_modes,
                'transformer': [None],
                'estimator': [RandomForestClassifier(n_estimators=250, class_weight=class_weight)],
            },
        ]
        scoring = {
            'precision': metrics.make_scorer(micro_average_precision_score),
            'accuracy': metrics.make_scorer(metrics.accuracy_score)
        }
        estimator = SVC()
        refit = 'precision'
    elif experiment_type == 'cluster':
        parameters = [ #CLUSTERING
            {
                'source__features': FEATURE_MODES,
                'source__scaling_type': SCALE_MODES,
                'transformer': [None, PCA(n_components=55)],
                'estimator': [KMeans()],
                'estimator__n_clusters': [2],
            },
        ]
        scoring = {
            'NMI': metrics.make_scorer(metrics.adjusted_mutual_info_score),
        }
        estimator = KMeans()
        refit = 'NMI'

    estimator = Pipeline([
        ('source', ClassificationExperiment()),
        ('scaler', StandardScaler()),
        ('transformer', PCA()),
        ('estimator', estimator)
    ])

    subset = STATS['class_ids'].apply(lambda x: True).as_matrix()
#     subset[::100] = True
    grid = GridSearchCV(
        estimator, parameters,
        cv=StratifiedKFold(n_splits=3, shuffle=True),
        scoring=scoring, refit=refit,
        n_jobs=12, return_train_score=True)
    grid.fit(
        np.arange(len(STATS))[subset],
        STATS['class_ids'].as_matrix()[subset])
    return pd.DataFrame(grid.cv_results_)

results = experiment('classification')

In [ ]:
results['PCA'] = results['param_transformer'].apply(lambda x: 0 if x is None else x.n_components)
results['model'] = results['param_estimator'].apply(lambda x: repr(x).split("(")[0])
results['param_estimator__C'] = results['param_estimator__C'].apply(lambda x: 0 if np.isnan(x) else x)
results[results['param_source__scaling_type'] == 'stretch'].sort_values('mean_test_accuracy', ascending=False).head(10)

In [ ]:
# results.to_csv("/data/acfr/collated/thesis/results.csv")
results2 = results.copy()

In [ ]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
#     print(results.groupby(
#         ['param_source__scaling_type', 'param_source__features',
#          'PCA', 'model', 'param_estimator__C']).mean()[['mean_test_accuracy']])

group = ['param_source__scaling_type', 'param_source__features', 'param_source__mask_mode', 'model']
idx = results.sort_values(by='mean_test_accuracy', ascending=False).index
results.loc[idx]#
# results.groupby(group)['mean_test_accuracy'].max()[24:]


In [ ]:
for idx, val in results.groupby(group[:-1], sort=True):
    idx_max_acc = val['mean_test_accuracy'].idxmax()
    idx_max_precision = val['mean_test_precision'].idxmax()
#     print(idx_max_acc == idx_max_precision)
    best = results.loc[idx_max_precision]
#     print(*idx, sep="\t", end='\t')
    print(
#         round(best['mean_test_accuracy'], 3),
#         round(best['mean_test_precision'], 3),
#         best['model'],
        best['param_estimator__C'], sep='\t')


In [ ]:
results.to_csv(prefix + "results_type.csv")

In [ ]:
group = ['param_source__scaling_type', 'param_source__features']
idx = results.groupby(group, sort=False)['mean_test_score'].idxmax()
results.loc[idx].sort_values(group)

In [ ]:
results = pd.concat([results, results2], axis=0)

In [ ]:
results = pd.read_csv("/data/acfr/collated/thesis/lettuce/pickle/lettuce-type_results_type.csv")